In [16]:
%matplotlib inline
import os
import pandas as pd
import re
import matplotlib.pyplot as plt
import pickle as pkl
import seaborn as sns
from utils.metrics import *

width = 6
height = 4
plt.figure(figsize=(width,height))
sns.set(rc={"figure.figsize":(width, height)})
root_baseline_data_path = '../output/%s/%s/%s/'
seq_len = 96

def get_borders(data_name, data_len):
    border1s = [0, 12 * 30 * 24 * 4 - seq_len, 12 * 30 * 24 * 4 + 4 * 30 * 24 * 4 - seq_len]
    border2s = [12 * 30 * 24 * 4, 12 * 30 * 24 * 4 + 4 * 30 * 24 * 4, 12 * 30 * 24 * 4 + 8 * 30 * 24 * 4]

    if data_name.find('ett') != -1:
        return border1s, border2s

    num_train = int(data_len * 0.7)
    num_test = int(data_len * 0.2)
    num_vali = data_len - num_train - num_test
    border1s = [0, num_train - seq_len, data_len - num_test - seq_len]
    border2s = [num_train, num_train + num_vali, data_len]

    return border1s, border2s

def get_raw_columns(columns):
    return [col for col in columns if 'R' in col]


def get_eb_columns(columns, eb):
    return [col for col in columns if col.endswith(f'E{eb}')]


def get_eb(exp_str):
    eb = re.findall('eb[_.0-9]+', exp_str)[0]
    eb = eb.replace('eb', '')
    eb = eb.replace('_', '')
    eb = float(eb)
    if eb >= 1:
        eb *= 0.01

    return eb


def load(path):
    with open(path, 'rb') as f:
        return pkl.load(f)


def metrics_ensemble(pred, true):
    mae = MAE(pred, true)
    rmse = RMSE(pred, true)
    rse = RSE(pred, true)
    nrmse = NRMSE(pred, true)
    corr = CORR(pred, true)
    psnr = PSNR(pred, true)

    return {'mae': mae,
            'rmse': rmse,
            'nrmse': nrmse,
            'rse': rse,
            'corr': corr,
            'psnr': psnr}


def load_pkl(path):
    with open(path, 'rb') as f:
        return pkl.load(f)


def get_baseline(model:str, data:str):
    raw_data_path = root_baseline_data_path%(model, data, 'raw') + 'true'
    raw_data = load(raw_data_path)

    predictions_path = root_baseline_data_path%(model, data, 'raw') + 'output'
    pred_data = load(predictions_path)
    results = metrics_ensemble(pred_data, raw_data)
    df = pd.DataFrame(results, index=[0.0])
    df.index.name = 'eb'
    return df


def get_forecasting_results(model: str, data_file: str, data_name: str,
                            eblc_name: str, target_ot: str, ebs_values: tuple):
    df = pd.read_parquet(f'../data/compressed/{eblc_name}/{data_file}')
    tmetrics = []
    ebs = []
    border1s, border2s = get_borders(data_name, len(df))
    raw_df = df[f'{target_ot}-R'].values[border1s[2]:border2s[2]]
    for eb in ebs_values:
        decomp_target_var = df[[f'{target_ot}-E{eb}']].values[border1s[2]:border2s[2]][:, 0]
        results = metrics_ensemble(decomp_target_var, raw_df)
        tmetrics.append(results)
        ebs.append(eb)

    df = pd.DataFrame(tmetrics)
    df['eb'] = ebs
    de = df.groupby(['eb']).median()

    raw_data_path = root_baseline_data_path%(model, data_name, 'raw') + 'true'
    raw_data = load(raw_data_path)

    tmetrics = []
    ebs = []
    for root, dr, files in os.walk(root_baseline_data_path%(model, data_name, eblc_name)+ 'predictions'):
        for file in files:
            results = metrics_ensemble(load(root+os.sep+file), raw_data)
            tmetrics.append(results)
            ebs.append(get_eb(file))

    df = pd.DataFrame(tmetrics)
    df['eb'] = ebs
    fr = df.groupby(['eb']).median()
    fr.sort_index(inplace=True)
    return fr, de


def get_solar_baseline_true():
    var_map = dict()
    all_results = list()
    all_true = dict()
    for root, dr, files in os.walk('../output/xgboost/solar/raw/'):
        for file in files:
            if 'hyperparameter' in file:
                continue
            t, var = file.strip('.pkl').split('_')
            if var in var_map:
                ifile = var_map[var]
                pf = load_pkl(f'../output/xgboost/solar/raw/{ifile}')
                nf = load_pkl(f'../output/xgboost/solar/raw/{file}')
                if 'true' in ifile:
                    all_true[var] = pf
                    results = metrics_ensemble(nf, pf)
                else:
                    all_true[var] = nf
                    results = metrics_ensemble(pf, nf)
                all_results.append(results)
            else:
                var_map[var] = file
    df = pd.DataFrame(all_results)
    df = df.mean().to_frame().T
    df.index._name = 'eb'
    return df, all_true

def get_forecasting_results_solar(model: str, eblc_name: str, ebs_values: tuple):
    df = pd.read_parquet(f'../data/compressed/{eblc_name}/solar_output_data_points.parquet')
    tmetrics = []
    ebs = []
    border1s, border2s = get_borders('solar', len(df))
    raw_columns = get_raw_columns(df.columns)
    raw_df = df[raw_columns].values[border1s[2]:border2s[2]]
    for eb in ebs_values:
        eb_columns = get_eb_columns(df.columns, eb)
        decomp_target_var = df[eb_columns].values[border1s[2]:border2s[2]]
        results = metrics_ensemble(decomp_target_var, raw_df)
        tmetrics.append(results)
        ebs.append(eb)

    df = pd.DataFrame(tmetrics)
    df['eb'] = ebs
    de = df.groupby(['eb']).mean()

    solar_baseline, raw_data_map = get_solar_baseline_true()
    all_results = []
    ebs = []
    # tvars = []
    for root, dr, files in os.walk(root_baseline_data_path%(model, 'solar', eblc_name)+ 'predictions'):
        for file in files:
            var = re.findall('v[0-9]+', file)[0]
            results = metrics_ensemble(load(root+os.sep+file), raw_data_map[var])
            all_results.append(results)
            ebs.append(get_eb(file))
            # tvars.append(var)

    df = pd.DataFrame(all_results)
    df['eb'] = ebs
    fr = df.groupby(['eb']).mean()
    fr.sort_index(inplace=True)
    return solar_baseline, fr, de


def get_transformation_error(data_file: str, data_name: str,
                            eblc_name: str, target_ot: str, ebs_values: tuple):
    df = pd.read_parquet(f'../data/compressed/{eblc_name}/{data_file}')
    tmetrics = []
    ebs = []
    border1s, border2s = get_borders(data_name, len(df))
    raw_df = df[f'{target_ot}-R'].values[border1s[2]:border2s[2]]
    for eb in ebs_values:
        decomp_target_var = df[[f'{target_ot}-E{eb}']].values[border1s[2]:border2s[2]][:, 0]
        results = metrics_ensemble(decomp_target_var, raw_df)
        tmetrics.append(results)
        ebs.append(eb)

    df = pd.DataFrame(tmetrics)
    df['eb'] = ebs
    return df


def concat_baseline_forecasting_result(br, fr, de):
    concat_forecasting_results = pd.concat([br, fr], axis=0)
    metric_indexed_results = pd.DataFrame()
    metric_indexed_results['error'] = [0.0]+list(de['nrmse'].values)
    metric_indexed_results['mae'] = concat_forecasting_results['mae'].values
    metric_indexed_results['rmse'] = concat_forecasting_results['rmse'].values
    metric_indexed_results['nrmse'] = concat_forecasting_results['nrmse'].values
    metric_indexed_results['rse'] = concat_forecasting_results['rse'].values
    metric_indexed_results['corr'] = concat_forecasting_results['corr'].values
    metric_indexed_results['data_corr'] = [1.0]+list(de['corr'].values)
    metric_indexed_results.set_index('error', inplace=True)
    metric_indexed_results['eb'] = [0.0]+list(de.index)
    return metric_indexed_results



<Figure size 432x288 with 0 Axes>

In [20]:
target_variables_map = {'ettm1':'OT', 'ettm2': 'OT', 'aus_electrical_demand': 'y', 'pweather': 'OT', 'wind': 'active power'}
bounds = [(0.01, 0.03, 0.05, 0.07, 0.10, 0.15, 0.20, 0.25, 0.30, 0.40, 0.50, 0.65, 0.8), (1, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 65, 80),
          (1.0, 1.3, 1.5, 1.7, 2.0, 2.3, 2.5, 2.7, 3.0, 5.0, 7.0, 10.0), (0.01, 0.013, 0.015, 0.017, 0.02, 0.023, 0.025, 0.027, 0.03, 0.05, 0.07, 0.1)]
all_results = pd.DataFrame()
for data in ['ettm1', 'ettm2', 'pweather', 'aus', 'solar', 'wind']:
    data_results = pd.DataFrame()
    for eblc in ['pmc', 'swing', 'sz']:
        print(eblc, data)
        if 'solar' in data:
            baseline_results, forecasting_results, dec_error = get_forecasting_results_solar(model='xgboost', eblc_name=eblc, ebs_values=bounds[0] if eblc == 'sz' else bounds[1])
        elif 'aus' in data:
            baseline_results = get_baseline('xgboost', 'aus')
            forecasting_results, dec_error = get_forecasting_results(model='xgboost',
                                                                     data_file=f'aus_electrical_demand_points.parquet',
                                                                     data_name='aus',
                                                                     eblc_name=eblc,
                                                                     target_ot='y',
                                                                     ebs_values=bounds[0] if eblc == 'sz' else np.asarray(bounds[1])*1.0)
        elif 'weather' in data:
            baseline_results = get_baseline('xgboost', data)
            forecasting_results, dec_error = get_forecasting_results(model='xgboost',
                                                                     data_file=f'{data}_output_data_points.parquet',
                                                                     data_name=data,
                                                                     eblc_name=eblc,
                                                                     target_ot=target_variables_map[data] if eblc == 'sz' else target_variables_map[data].replace(' ', '_'),
                                                                     ebs_values=bounds[3] if eblc == 'sz' else bounds[2])
        elif 'wind' in data:
            baseline_results = get_baseline('xgboost', data)
            forecasting_results, dec_error = get_forecasting_results(model='xgboost',
                                                                     data_file=f'{data}_output_data_points.parquet',
                                                                     data_name=data,
                                                                     eblc_name=eblc,
                                                                     target_ot=target_variables_map[data] if eblc == 'sz' else target_variables_map[data].replace(' ', '_'),
                                                                     ebs_values=bounds[0] if eblc == 'sz' else bounds[1])
        else:
            baseline_results = get_baseline('xgboost', data)
            forecasting_results, dec_error = get_forecasting_results(model='xgboost',
                                                                     data_file=f'{data}_output_data_points.parquet',
                                                                     data_name=data,
                                                                     eblc_name=eblc,
                                                                     target_ot=target_variables_map[data] if eblc == 'sz' else target_variables_map[data].replace(' ', '_'),
                                                                     ebs_values=bounds[0] if eblc == 'sz' else bounds[1])

        concatenated = concat_baseline_forecasting_result(baseline_results, forecasting_results, dec_error)
        concatenated['eblc'] = eblc
        concatenated.at[0, 'eblc'] = 'baseline'
        data_results = pd.concat([data_results, concatenated])
        data_results.drop_duplicates(inplace=True)

    data_results['data'] = data
    all_results = pd.concat([all_results, data_results])

all_results.to_csv('../results/tfe/per_model/xgboost_results.csv')

pmc ettm1
swing ettm1
sz ettm1
pmc ettm2
swing ettm2
sz ettm2
pmc pweather
swing pweather
sz pweather
pmc aus
swing aus
sz aus
pmc solar
swing solar
sz solar
pmc wind
swing wind
sz wind


In [13]:
baseline = all_results[all_results.eb == 0]
baseline['order'] = [1,2,4 , 6, 3, 5]
baseline.sort_values('order')

D:\Anaconda\envs\dart\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,mae,rmse,nrmse,rse,corr,data_corr,eb,eblc,data,order
error,,,,,,,,,,
0.0,0.097972,0.135575,0.067220,0.395499,0.931523,1.0,0.0,baseline,ettm1,1
0.0,0.135144,0.208168,0.061986,0.338052,0.946065,1.0,0.0,baseline,ettm2,2
0.0,0.218851,0.387294,0.124597,0.453802,0.902243,1.0,0.0,baseline,solar,3
0.0,0.288870,0.448702,0.076173,0.500371,0.868724,1.0,0.0,baseline,pweather,4
0.0,0.088427,0.147443,0.032564,0.173707,0.984768,1.0,0.0,baseline,wind,5
0.0,0.190345,0.291954,0.047381,0.337692,0.949146,1.0,0.0,baseline,aus,6


In [15]:
all_results[all_results.data == 'solar']

,mae,rmse,nrmse,rse,corr,data_corr,eb,eblc,data
error,,,,,,,,,
0.000000,0.218851,0.387294,0.124597,0.453802,0.902243,1.000000,0.00,baseline,solar
0.000514,0.001021,0.004081,0.001313,0.004774,0.999989,0.999989,1.00,pmc,solar
0.001879,0.005228,0.015111,0.004851,0.017669,0.999844,0.999844,3.00,pmc,solar
0.003164,0.009748,0.025343,0.008133,0.029623,0.999561,0.999562,5.00,pmc,solar
0.004328,0.014026,0.034605,0.011111,0.040457,0.999181,0.999182,7.00,pmc,solar
0.005910,0.020012,0.047303,0.015185,0.055297,0.998469,0.998471,10.00,pmc,solar
0.008183,0.028597,0.065422,0.020981,0.076441,0.997072,0.997073,15.00,pmc,solar
0.010110,0.035801,0.081173,0.026018,0.094840,0.995486,0.995486,20.00,pmc,solar
0.011745,0.041908,0.094458,0.030268,0.110365,0.993880,0.993879,25.00,pmc,solar
